In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import scipy.io as sio

import torchvision

import scipy.io as sio

In [ ]:
from utils import f_rescale_dataset, f_train_test_split, f_get_mask, f_get_all_masks, f_Residuals
# from utils import f_reshape_training_data, f_rotate, f_translate, f_resize, f_augment_img, f_augment_dataset
from utils import f_reshape_training_data, f_rotate_and_zoom, f_random_crop, f_rotate_and_zoom_all, f_crop_all, f_flip_all, f_augment_dataset2

In [ ]:
N = 4
batch_size = N
ngpus = 1
num_workers = 1

In [ ]:
def f_normalize_imgs(imgs):
    v, i = torch.max(imgs,1)
    v, i = torch.max(v,1)

    return (imgs.T/v).T

In [ ]:
# import scipy.io as sio

# %cd /Users/Pracioppo/Desktop/VPTR/GA

# GA_data_FAF = sio.loadmat('GA_data_orig_256.mat')["GA_data"].astype(np.float32)
# GA_data_mask = sio.loadmat('GA_data_256.mat')["GA_data_256"].astype(np.float32)

# print(np.shape(GA_data_FAF))
# print(np.shape(GA_data_mask))

In [ ]:
# %cd Y:/FAZ/Healthy/all_Healthy/extras/mats

# imgs_DVC = torch.from_numpy(sio.loadmat('DVC_resized.mat')["Imgs"].astype(np.float32))
# masks_DVC = torch.from_numpy(sio.loadmat('DVC_masks.mat')["Imgs"].astype(np.float32))
# imgs_SVC = torch.from_numpy(sio.loadmat('SVC_resized.mat')["Imgs"].astype(np.float32))
# masks_SVC = torch.from_numpy(sio.loadmat('SVC_masks.mat')["Imgs"].astype(np.float32))

### Photocoagulation

In [ ]:
# %cd Y:\FAZ\Photocoagulation\Laser and Mask - All-orig-extractedImages\mats

# imgs = torch.from_numpy(sio.loadmat('resized.mat')["Imgs_rsz"].astype(np.float32))
# masks = torch.from_numpy(sio.loadmat('real_masks.mat')["Imgs_mask"].astype(np.float32))
# masks /= torch.max(masks)
# L = len(imgs)
# L

# imgs = f_normalize_imgs(imgs)
# masks = f_normalize_imgs(masks)

### Healthy

In [ ]:
%cd Y:\FAZ\FAZ-cropped-SVC-graded\Healthy\all_Healthy\PR\mats

### ALZ

In [ ]:
# %cd Y:\FAZ\FAZ-cropped-SVC-graded\Alz\all_Alz\PR\mats

### AMD

In [ ]:
# %cd Y:\FAZ\FAZ-cropped-SVC-graded\AMD\all_AMD\PR\mats

### DR

In [ ]:
# %cd Y:\FAZ\FAZ-cropped-SVC-graded\DR\all_DR\PR\mats

In [ ]:
imgs_DVC = torch.from_numpy(sio.loadmat('DVC_resized.mat')["Imgs"].astype(np.float32))
masks_DVC = torch.from_numpy(sio.loadmat('DVC_fake_masks.mat')["Imgs"].astype(np.float32))
imgs_SVC = torch.from_numpy(sio.loadmat('SVC_resized.mat')["Imgs"].astype(np.float32))
# masks_SVC = torch.from_numpy(sio.loadmat('SVC_fake_masks.mat')["Imgs"].astype(np.float32))
masks_SVC = torch.from_numpy(sio.loadmat('SVC_real_masks_aya.mat')["Imgs"].astype(np.float32))

masks_DVC /= torch.max(masks_DVC)
masks_SVC /= torch.max(masks_SVC)
imgs_SVC /= torch.max(imgs_SVC)

masks_DVC = 1.0*(masks_DVC > 0.1)
masks_SVC = 1.0*(masks_SVC > 0.1)

L = len(imgs_DVC)
L
len(masks_SVC)

In [ ]:
for i in np.arange(L):    
#     plt.imshow(255*masks_DVC[i,:,:] + imgs_DVC[i,:,:])
#     plt.show()
    plt.imshow(255*masks_SVC[i,:,:])
    plt.show()
    plt.imshow(255*masks_SVC[i,:,:] + 255*imgs_SVC[i,:,:])
    plt.show()

# for i in np.arange(L):   
#     plt.imshow(255*imgs[i,:,:])
#     plt.show()
#     plt.imshow(255*masks[i,:,:] + 255*imgs[i,:,:])
#     plt.show()

In [ ]:
# def f_augment_all(imgs, masks, p_rot = 0.8, p_flip = 0.5, p_jitter = 0.8, p_shear = 0.25, N = 10):

#     VF = torchvision.transforms.RandomVerticalFlip(1)
#     CJ = torchvision.transforms.ColorJitter(brightness = 0.8, contrast = 0.8)

#     Imgs = torch.zeros(N*L,256,256)
#     Masks = torch.zeros(N*L,256,256)

#     for j in np.arange(N):

#         for i in np.arange(L):

#             img = imgs[i]
#             mask = masks[i]

#             img /= torch.max(img)

#             if j > 0:

#                 if np.random.uniform(0,1) < p_shear:
#                     shr = np.random.uniform(-25,25)
#                     RA = torchvision.transforms.RandomAffine(degrees=0,shear=(shr,shr))
#                     img = RA.forward(img.unsqueeze(0)).squeeze()
#                     mask = RA.forward(mask.unsqueeze(0)).squeeze()

#                 if np.random.uniform(0,1) < p_rot:
#                     angle = np.random.uniform(0,90)
#                     img = f_rotate_and_zoom(img, angle)
#                     mask = f_rotate_and_zoom(mask, angle)

#                 if np.random.uniform(0,1) < p_flip:
#                     img = VF.forward(img)
#                     mask = VF.forward(mask)

#                 if np.random.uniform(0,1) < p_jitter:
#                     img = CJ.forward(img.unsqueeze(0)).squeeze()

#                 mask = mask>0.5

#             Imgs[i*N + j,:,:] = img
#             Masks[i*N + j,:,:] = mask

#     return Imgs, Masks

In [ ]:
def f_augment_all2(imgs_svc, masks_svc, imgs_dvc, masks_dvc, p_rot = 0.8, p_flip = 0.5, p_jitter = 0.8, p_shear = 0.25, N = 10):

    VF = torchvision.transforms.RandomVerticalFlip(1)
    HF = torchvision.transforms.RandomHorizontalFlip(1)
    CJ = torchvision.transforms.ColorJitter(brightness = 0.8, contrast = 0.8)

    Imgs_SVC = torch.zeros(N*L,256,256)
    Imgs_DVC = torch.zeros(N*L,256,256)
    Masks_SVC = torch.zeros(N*L,256,256)
    Masks_DVC = torch.zeros(N*L,256,256)

    for j in np.arange(N):

        for i in np.arange(L):

            img_svc = imgs_svc[i]
            mask_svc = masks_svc[i]
            img_dvc = imgs_dvc[i]
            mask_dvc = masks_dvc[i]

            img_svc /= torch.max(img_svc)
            img_dvc /= torch.max(img_dvc)

            if j > 0:

                if np.random.uniform(0,1) < p_flip:
                    img_svc = VF.forward(img_svc)
                    mask_svc = VF.forward(mask_svc)
                    img_dvc = VF.forward(img_dvc)
                    mask_dvc = VF.forward(mask_dvc)
                    
                if np.random.uniform(0,1) < p_flip:
                    img_svc = HF.forward(img_svc)
                    mask_svc = HF.forward(mask_svc)
                    img_dvc = HF.forward(img_dvc)
                    mask_dvc = HF.forward(mask_dvc)
                    
                if np.random.uniform(0,1) < p_rot:
                    angle = np.random.uniform(0,90)
                    img_svc = f_rotate_and_zoom(img_svc, angle)
                    mask_svc = f_rotate_and_zoom(mask_svc, angle)
                    img_dvc = f_rotate_and_zoom(img_dvc, angle)
                    mask_dvc = f_rotate_and_zoom(mask_dvc, angle)
                    
                if np.random.uniform(0,1) < p_shear:
                    shr = np.random.uniform(-25,25)
                    RA = torchvision.transforms.RandomAffine(degrees=0,shear=(shr,shr))
                    img_svc = RA.forward(img_svc.unsqueeze(0)).squeeze()
                    mask_svc = RA.forward(mask_svc.unsqueeze(0)).squeeze()
                    img_dvc = RA.forward(img_dvc.unsqueeze(0)).squeeze()
                    mask_dvc = RA.forward(mask_dvc.unsqueeze(0)).squeeze()

                if np.random.uniform(0,1) < p_jitter:
                    img_svc = CJ.forward(img_svc.unsqueeze(0)).squeeze()
                    img_dvc = CJ.forward(img_dvc.unsqueeze(0)).squeeze()

                mask_svc = mask_svc>0.5
                mask_dvc = mask_dvc>0.5

            Imgs_SVC[i*N + j,:,:] = img_svc
            Masks_SVC[i*N + j,:,:] = mask_svc
            Imgs_DVC[i*N + j,:,:] = img_dvc
            Masks_DVC[i*N + j,:,:] = mask_dvc

    return Imgs_SVC, Masks_SVC, Imgs_DVC, Masks_DVC

In [ ]:
def f_augment_all_single(imgs, masks, p_rot = 0.5, p_flip = 0.5, p_jitter = 0.8, p_shear = 0.5, N = 1):

#     np.random.seed(seed=None)
    
    VF = torchvision.transforms.RandomVerticalFlip(1)
    HF = torchvision.transforms.RandomHorizontalFlip(1)
    CJ = torchvision.transforms.ColorJitter(brightness = 0.8, contrast = 0.8)

    Imgs = torch.zeros(N*L,256,256)
    Masks = torch.zeros(N*L,256,256)

    for j in np.arange(N):

        for i in np.arange(L):

            img = imgs[i]
            mask = masks[i]

#             if j > 0:
            
            h_flip_dice = np.random.uniform(0,1)
            if h_flip_dice < p_flip:
                img = HF.forward(img)
                mask = HF.forward(mask)

            rot_dice = np.random.uniform(0,1)
            if rot_dice < p_rot:
#                     angle = np.random.uniform(0,90)
#                     angle = np.random.uniform(0,90**0.5/2)**2
                angle = np.random.uniform(0,1)**2*45
                img = f_rotate_and_zoom(img, angle)
                mask = f_rotate_and_zoom(mask, angle)

            v_flip_dice = np.random.uniform(0,1)
            if v_flip_dice < p_flip:
                img = VF.forward(img)
                mask = VF.forward(mask)

            shear_dice = np.random.uniform(0,1)
            if shear_dice < p_shear:
                shr = np.random.uniform(-15,15)

                if rot_dice > p_rot:
                    transx = np.random.uniform(-50,50)
                    transy = np.random.uniform(-50,50)
                else:
                    transx = 0
                    transy = 0
                # RA = torchvision.transforms.RandomAffine(degrees=0,translate=(transx,transy),shear=(shr,shr))
#                     img = RA.forward(img.unsqueeze(0)).squeeze()
#                     mask = RA.forward(mask.unsqueeze(0)).squeeze()
                img = torchvision.transforms.functional.affine(img.unsqueeze(0),angle=0,scale=1,translate=(transx,transy),shear=(shr,shr))
                mask = torchvision.transforms.functional.affine(mask.unsqueeze(0),angle=0,scale=1,translate=(transx,transy),shear=(shr,shr))

                jitter_dice = np.random.uniform(0,1)
                if jitter_dice < p_jitter:
                    img = CJ.forward(img.unsqueeze(0)).squeeze()
            
#             print(h_flip_dice)
#             print(rot_dice)
#             print(v_flip_dice)
#             print(shear_dice)
#             print(jitter_dice)
            
#             mask = mask>0.5

            Imgs[i*N + j,:,:] = img
            Masks[i*N + j,:,:] = mask

    return Imgs, Masks

In [ ]:
# Imgs_SVC, Masks_SVC, Imgs_DVC, Masks_DVC = f_augment_all2(imgs_SVC, masks_SVC, imgs_DVC, masks_DVC)

# if torch.sum(torch.isnan(Imgs_SVC + Masks_SVC + Imgs_DVC + Masks_DVC)) > 0:
#     print('ERROR')

In [ ]:
# for i in np.arange(N*L):
#     plt.imshow((Imgs_SVC + Masks_SVC)[i,:,:])
#     plt.show()
# #     plt.imshow((Imgs_DVC + Masks_DVC)[i,:,:])
# #     plt.show()

In [ ]:
# imgs_SVC.size()

In [ ]:
Imgs, Masks = f_augment_all_single(imgs_SVC, masks_SVC, p_rot = 0.5, p_flip = 0.5, p_jitter = 0.8, p_shear = 0.5, N = 1)

if torch.sum(torch.isnan(Imgs+Masks)) > 0:
    print('ERROR')

for i in np.arange(N*L):
    plt.imshow((255*Imgs)[i,:,:])
    plt.imshow((255*Imgs + 255*Masks)[i,:,:])
    plt.show()

In [ ]:
sio.savemat('Imgs_DVC_aug.mat',{"Imgs_DVC": Imgs_DVC.numpy()})
sio.savemat('Imgs_SVC_aug.mat',{"Imgs_SVC": Imgs_SVC.numpy()})
sio.savemat('Fake_Masks_DVC_aug.mat',{"Masks_DVC": Masks_DVC.numpy()})
# sio.savemat('Fake_Masks_SVC_aug.mat',{"Masks_SVC": Masks_SVC.numpy()})
sio.savemat('Real_Masks_SVC_aug.mat',{"Masks_SVC": Masks_SVC.numpy()})

In [ ]:
# sio.savemat('Imgs_aug.mat',{"Imgs": Imgs.numpy()})
# sio.savemat('Fake_Masks_aug.mat',{"Masks": Masks.numpy()})

In [ ]:
print("Done")